# Software-Defined Networks Emulation in Mininet

Mininet is a network emulator that allows for rapid development of vast network architectures. Its limits lie with the hardware limits of the platform that runs the mininet. In this course, we will discuss some basic usage of mininet to emulate various network topologies.

**IMPORTANT:** Mininet's *mn* wrapper works as a CLI so we cannot use it in the Jupyter. Instead, please use the remote console to the servers having the mininet package installed and ready. These servers are at:

You can log in using standard ssh connection. From the terminal in your computer please use the following command.

```bash
ssh student@10.100.0.XX
ssh student@158.196.244.134 -p XX22  

XX: 41-53
```
The password for the connection is **student**.

When you successfully log in, please elevate your permission level to root user, since mininet can only be used by root users.

```bash
su -
```
Password is again **student**.

## Running Mininet

To define a custom topology, you can use a sceleton class in the cell below.

In [ ]:
from mininet.net import Mininet
from mininet.topo import Topo
from mininet.node import OVSSwitch, Controller
from mininet.link import TCLink

class CustomTopology(Topo):
    def build(self):
        # Add switches
        s1 = self.addSwitch('s1')
        s2 = self.addSwitch('s2')

        # Add hosts
        h1 = self.addHost('h1', ip='10.0.0.1/24')
        h2 = self.addHost('h2', ip='10.0.0.2/24')
        h3 = self.addHost('h3', ip='10.0.0.3/24')
        h4 = self.addHost('h4', ip='10.0.0.4/24')

        # Add links with QoS parameters
        self.addLink(h1, s1, bw=1000, delay='1ms', loss=0, max_queue_size=1000, use_htb=True)
        self.addLink(h2, s1, bw=1000, delay='1ms', loss=0, max_queue_size=1000, use_htb=True)
        self.addLink(h3, s2, bw=1000, delay='1ms', loss=0, max_queue_size=1000, use_htb=True)
        self.addLink(h4, s2, bw=1000, delay='1ms', loss=0, max_queue_size=1000, use_htb=True)
        self.addLink(s1, s2, bw=1000, delay='5ms', loss=0, max_queue_size=1000, use_htb=True)

def start_mininet():
    topo = CustomTopology()
    net = Mininet(topo=topo, switch=OVSSwitch, link=TCLink, controller=Controller)
    net.start()

    # Set QoS parameters for hosts
    for host in net.hosts:
        host.cmd('tc qdisc add dev {}-eth0 root handle 1: htb default 11'.format(host.name))
        host.cmd('tc class add dev {}-eth0 parent 1: classid 1:1 htb rate 1000mbit'.format(host.name))
        host.cmd('tc class add dev {}-eth0 parent 1:1 classid 1:11 htb rate 1000mbit'.format(host.name))
        host.cmd('tc qdisc add dev {}-eth0 parent 1:11 handle 10: netem delay 5ms'.format(host.name))

    net.interact()

if __name__ == '__main__':
    start_mininet()


**tc qdisc add**

handle 1:: Assigns a unique identifier (handle) to this qdisc. The 1: is the major identifier, and it uniquely identifies this qdisc in the system.

htb: Specifies the Hierarchical Token Bucket algorithm for bandwidth management.

default 11: Sets the default class for traffic that doesn't match any specific class to class 1:11.

**tc class add**

parent 1:: Indicates that this class is a child of the qdisc with the handle 1:.

classid 1:1: Assigns a unique identifier to this class within the qdisc. The major identifier is 1: and the minor identifier is 1.

htb: Specifies the use of the Hierarchical Token Bucket algorithm.

**The structure includes a root qdisc with a handle of 1:, a class with classid 1:1 as a child of the root qdisc, and another qdisc as a child of the class with classid 1:11. The netem command is then used to introduce a delay of 5 milliseconds to the outgoing packets of this last qdisc.**



To run this topology, you can use the following command, which assumes that the file with the topology is called **ex.py**. **Important** for the network parameters to work as expected, **the link type must be set to tc**.

```bash
mn --custom=ex.py --topo mytopo --mac --link tc
```

### Task

* Add network loss to the topology as well.
* Measure the network performance using iperf tool.